# Calculate mangrove gain and loss from mangrove extent

Based on mangrove extent layer. Cureent version is v3

In [1]:
import ee
import geemap
%run utils.ipynb

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

Enter verification code: 4/1AdQt8qiHnk6AZ00WR3OdJjv27SOoMMHapkgMbsLpluV1RdhRG-NjmKgVwhQ

Successfully saved authorization token.


In [3]:

ee.Initialize()

### Create GEE Assets for Gain

In [4]:
# variables
data_year_range = [1996, 2007, 2008, 2009, 2010, 2015, 2016, 2017, 2018, 2019, 2020]
data_version = 'v3'
extent = 'mangrove_extent'
asset = {
    'gain': f'projects/global-mangrove-watch/land-cover/{extent}_gain-{data_version}',
    'loss': f'projects/global-mangrove-watch/land-cover/{extent}_loss-{data_version}'
}

# DO EXPORT
# GET DATA LAYERS
land_cover = ee.ImageCollection(f'projects/global-mangrove-watch/land-cover/{extent}-{data_version}')

# SET EXPORT REGION
region = ee.Geometry.Polygon([-180, 33, 0,33,180, 33,180,-40,0,-40,-180,-40], None, False)

In [5]:
def create_gain_loss_asset(data_version, year_range, keyword):
    """
    Create an asset with the gain/loss of the mangrove extent
    """
    dataset = f'{extent}_{keyword}'
    ee_asset_path = f'projects/global-mangrove-watch/land-cover/{dataset}-{data_version}'
    
    description = f"""
    # Global {keyword} in extent of mangrove forests

    ## Methodology

    This dataset shows the global gain extent of mangroves for the time intervals 
    {', '.join([f'{x}--{y}' for x, y in zip(year_range[:-1], year_range[1:])])}. Where gain is defined as the 
    difference between mangrove habitat extent layers X2 and X1 (X2-X1) from consecutive time points 1 and 2.  

    The dataset has been generated by vizzuality for Global Mangrove Watch (GMW) a collaboration between Aberystwyth 
    University (U.K.), solo Earth Observation (soloEO), Wetlands International, the World Conservation Monitoring 
    Centre (UNEP-WCMC) and the Japan Aerospace Exploration Agency (JAXA). DOB Ecology supported the mapping in Africa 
    through the Mangrove Capital Africa project.  

    For the full documentation, please see the source [methodology](https://www.mdpi.com/2072-4292/10/10/1669).
    """

    collection_properties = ImageCollectionProperties(
        name = f"Global extent of mangrove forests ({keyword}) ({data_version})",
        version = data_version,
        creator = "Global Mangrove Watch (GMW): Aberystwyth University/soloEO/Wetlands International/UNEP-WCMC/JAXA/DOB Ecology",
        description = description,
        identifier = "",
        keywords = "Erosion; Coasts; Natural Infrastructure; Biodiversity; Blue Carbon; Forests; Mangroves; Landcover",
        citation = "Bunting, Pete, Ake Rosenqvist, Richard M. Lucas, Lisa-Maria Rebelo, Lammert Hilarides, Nathan Thomas, \
            Andy Hardy, Takuya Itoh, Masanobu Shimada, and C. Max Finlayson. “The Global Mangrove Watch—A New 2010 Global \
                Baseline of Mangrove Extent.” Remote Sensing 10, no. 10 (October 2018): 1669. doi: 10.3390/rs10101669.",
        license = "https://creativecommons.org/licenses/by/4.0/",
        url = "",
        language = "en", 
        altName = "Global Mangrove Watch, Version 3.0",
        distribution = "",
        variableMeasured = "Gain of mangrove forest habitat",
        units = "1",
        spatialCoverage = "Global tropics",
        temporalCoverage = ','.join(map(str, year_range)),
        dataLineage = "Raster data supplied by Aberystwyth University (Dr. Dave Bunting) as tilesets per year, \
            each tilset was combined, and added to Google earth engine as multi-temporal ImageCollection."
    )

    return ee.data.createAsset({"type":"ImageCollection"}, ee_asset_path, collection_properties.dict(exclude_none=True))

In [6]:
# TODO: rework this in a more component-based way
# CALCULATIONS
def export_diff(ic, s_year, e_year, calc_type, asset_ic_name, 
                region, pyramiding: str = 'MEAN', band_name: str= 'lc', 
                view = True, export = False):
    # Make time-interval label
    ti = f"{s_year}--{e_year}"
    # Create name string
    nm = f"gl_{str(s_year)}_{str(e_year)}_{calc_type}"
    # Make time_start
    ts = ee.Date.fromYMD(e_year, 1, 1)

    # Given a period of T1--T2
    # Set the order of subtraction depending on gain (T2-T1) or loss (T1-T2)
    if calc_type == 'gain':
        ts_x1 = ee.Date.fromYMD(e_year, 1, 1)
        ts_x2 = ee.Date.fromYMD(s_year, 1, 1)
        color = "lightgreen" 
    if calc_type == 'loss':
        ts_x1 = ee.Date.fromYMD(s_year, 1, 1)
        ts_x2 = ee.Date.fromYMD(e_year, 1, 1)
        color = "orange" 
        
    # Get the images and unmask 
    x1 = (ic.filterDate(ts_x1).first().unmask())
    x2 = (ic.filterDate(ts_x2).first().unmask())

    # Calculate difference and update mask
    diff = (x1.subtract(x2).selfMask()
    .set({
        'system:time_start': ts,
        'time_interval': ti,
        'start_year': s_year,
        'end_year': e_year,
        'change_type': calc_type  
    })).rename([band_name])

    # Get nominal scale
    ns = x1.projection().nominalScale().getInfo()

    # Create export params
    params = {
        'image': diff,
        'description': f"export_{nm}",
        'assetId': f"{asset_ic_name}/{nm}",
        'pyramidingPolicy':{band_name: pyramiding},
        'scale': ns,
        'crs': 'EPSG:4326',
        'region': region,
        'maxPixels': 1e13
        }

    # Optionally create export task
    task = ee.batch.Export.image.toAsset(**params)
    if export == True:
        # image_properties = ImageProperties(
        #             band_nodata_values = 0,
        #             band_pyramiding_policies = pyramiding,
        #             band_names = band_name,
        #             year = e_year,
        #         )
        # ee.data.createAsset({"type":"Image"}, f"{asset_ic_name}/{nm}", image_properties.dict(exclude_none=True))
        task.start()

    if view == True:
        print("Map:")
        # Create a geemap  object.
        Map = geemap.Map(center=(-7.998, 39.4767), zoom=9, basemap='HYBRID')
        vis_params = {
            'min': 1,
            'max': 1,
            'palette': ['eb6240'],
        }
        Map.addLayer(diff, vis_params, calc_type, True, 0.5)
        display(Map)

    return task


In [7]:
create_gain_loss_asset( data_version, data_year_range, 'gain')

EEException: Cannot overwrite asset 'projects/earthengine-legacy/assets/projects/global-mangrove-watch/land-cover/mangrove_extent_gain-v3'.

In [19]:
create_gain_loss_asset( data_version, data_year_range, 'loss')

{'type': 'IMAGE_COLLECTION',
 'name': 'projects/earthengine-legacy/assets/projects/global-mangrove-watch/land-cover/mangrove_extent_loss-v3',
 'id': 'projects/global-mangrove-watch/land-cover/mangrove_extent_loss-v3',
 'properties': {'altName': 'Global Mangrove Watch, Version 3.0',
  'citation': 'Bunting, Pete, Ake Rosenqvist, Richard M. Lucas, Lisa-Maria Rebelo, Lammert Hilarides, Nathan Thomas,             Andy Hardy, Takuya Itoh, Masanobu Shimada, and C. Max Finlayson. “The Global Mangrove Watch—A New 2010 Global                 Baseline of Mangrove Extent.” Remote Sensing 10, no. 10 (October 2018): 1669. doi: 10.3390/rs10101669.',
  'creator': 'Global Mangrove Watch (GMW): Aberystwyth University/soloEO/Wetlands International/UNEP-WCMC/JAXA/DOB Ecology',
  'dataLineage': 'Raster data supplied by Aberystwyth University (Dr. Dave Bunting) as tilesets per year,             each tilset was combined, and added to Google earth engine as multi-temporal ImageCollection.',
  'description': '

### Create GEE Assets for Loss

In [ ]:
# Loop through time-intervals
for t in asset.keys():
    for start, end in zip(data_year_range[:-1], data_year_range[1:]):
        export_diff(land_cover, start, end, t, asset[t], region,  view = False,  export = True)


In [40]:
# Check task status
!earthengine task list | head -n 20

Q6KQD66V5QCFMYBN7QIKO42J  Export.image  export_gl_2019_2020_loss                    READY      ---
BVLGMN5LIJD7FOM64JE3EZRN  Export.image  export_gl_2018_2019_loss                    READY      ---
B6DSZNIDSMXW34QI7DXLUVJ5  Export.image  export_gl_2017_2018_loss                    READY      ---
QVFDRJTRZ7R7ZJ2D5XSC45HS  Export.image  export_gl_2016_2017_loss                    READY      ---
KPJDTZRBREXD2YNBLBVGWN27  Export.image  export_gl_2015_2016_loss                    READY      ---
5M3ZKFOZV6MEHFJIVK5UPW25  Export.image  export_gl_2010_2015_loss                    READY      ---
OB4BILI3A7APEXENMIHCXG55  Export.image  export_gl_2009_2010_loss                    READY      ---
JRZQJLALP4FB2W5H3L5XETU4  Export.image  export_gl_2008_2009_loss                    READY      ---
76FKPVNHCI5WONA3TJ7X25VR  Export.image  export_gl_2007_2008_loss                    READY      ---
XQGMFGNLMG424GQBUJSODWXE  Export.image  export_gl_1996_2007_loss                    READY      ---
B4SPL5CUIR